# Pratice Exercise for SparkSession
## sample.csv, 1342-0.txt

In [9]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

# tag::ch01-code-example[]

(
    spark.read.csv("/sample.csv", header=True)
    .withColumn(
        "new_column", F.when(F.col("old_column") > 10, 10).otherwise(0)
    )
    .where("old_column > 8")
    .groupby("new_column")
    .count()
    .write.csv("/updated_frequencies.csv", mode="overwrite")
)

In [11]:
"""Book code for chapter 2 of Data Analysis with Python and PySpark"""

# tag::ch02-sparksession[]

from pyspark.sql import SparkSession  # <1>

spark = (SparkSession
         .builder  # <2>
         .appName("Analyzing the vocabulary of Pride and Prejudice.")  # <3>
         .getOrCreate())

# Move the file from local VM to hdfs
# Load the data from hadoop
book = spark.read.text("/user1/1342-0.txt")

book

DataFrame[value: string]

In [21]:
book.printSchema()

print(book.dtypes)

book.show()

root
 |-- value: string (nullable = true)

[('value', 'string')]
+--------------------+
|               value|
+--------------------+
|The Project Guten...|
|                    |
|This eBook is for...|
|almost no restric...|
|re-use it under t...|
|with this eBook o...|
|                    |
|                    |
|Title: Pride and ...|
|                    |
| Author: Jane Austen|
|                    |
|Posting Date: Aug...|
|Release Date: Jun...|
|Last Updated: Mar...|
|                    |
|   Language: English|
|                    |
|Character set enc...|
|                    |
+--------------------+
only showing top 20 rows



In [27]:
book.show(10, truncate = 50)

+--------------------------------------------------+
|                                             value|
+--------------------------------------------------+
|The Project Gutenberg EBook of Pride and Prejud...|
|                                                  |
|This eBook is for the use of anyone anywhere at...|
|almost no restrictions whatsoever.  You may cop...|
|re-use it under the terms of the Project Gutenb...|
|    with this eBook or online at www.gutenberg.org|
|                                                  |
|                                                  |
|                        Title: Pride and Prejudice|
|                                                  |
+--------------------------------------------------+
only showing top 10 rows



In [22]:
from pyspark.sql.functions import split

lines = book.select(split(book.value, " ").alias("line"))

lines.show(5)

book.select(book.value)

+--------------------+
|                line|
+--------------------+
|[The, Project, Gu...|
|                  []|
|[This, eBook, is,...|
|[almost, no, rest...|
|[re-use, it, unde...|
+--------------------+
only showing top 5 rows



DataFrame[value: string]

In [29]:
from pyspark.sql.functions import col

book.select(book.value)
book.select(book["value"])
book.select(col("value"))
book.select("value")

from pyspark.sql.functions import col, split

lines = book.select(split(col("value"), " "))

lines

# DataFrame[split(value,  , -1): array<string>]

lines.printSchema()

lines.show(5)

root
 |-- split(value,  , -1): array (nullable = true)
 |    |-- element: string (containsNull = true)

+--------------------+
| split(value,  , -1)|
+--------------------+
|[The, Project, Gu...|
|                  []|
|[This, eBook, is,...|
|[almost, no, rest...|
|[re-use, it, unde...|
+--------------------+
only showing top 5 rows



In [24]:
book.select(split(col("value"), " ")).printSchema()
# root
#  |-- split(value,  , -1): array (nullable = true)  <1>
#  |    |-- element: string (containsNull = true)

book.select(split(col("value"), " ").alias("line")).printSchema()

# root
#  |-- line: array (nullable = true) <2>
#  |    |-- element: string (containsNull = true)

# This looks a lot cleaner
lines = book.select(split(book.value, " ").alias("line"))

# This is messier, and you have to remember the name PySpark assigns automatically
lines = book.select(split(book.value, " "))
lines = lines.withColumnRenamed("split(value,  , -1)", "line")

root
 |-- split(value,  , -1): array (nullable = true)
 |    |-- element: string (containsNull = true)

root
 |-- line: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [17]:
from pyspark.sql.functions import explode, col

words = lines.select(explode(col("line")).alias("word"))

words.show(15)

+----------+
|      word|
+----------+
|       The|
|   Project|
| Gutenberg|
|     EBook|
|        of|
|     Pride|
|       and|
|Prejudice,|
|        by|
|      Jane|
|    Austen|
|          |
|      This|
|     eBook|
|        is|
+----------+
only showing top 15 rows



In [18]:
from pyspark.sql.functions import lower
words_lower = words.select(lower(col("word")).alias("word_lower"))

words_lower.show()

+----------+
|word_lower|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
|     pride|
|       and|
|prejudice,|
|        by|
|      jane|
|    austen|
|          |
|      this|
|     ebook|
|        is|
|       for|
|       the|
|       use|
|        of|
|    anyone|
+----------+
only showing top 20 rows



In [19]:
from pyspark.sql.functions import regexp_extract
words_clean = words_lower.select(
    regexp_extract(col("word_lower"), "[a-z]+", 0).alias("word")  # <1>
)

words_clean.show()

+---------+
|     word|
+---------+
|      the|
|  project|
|gutenberg|
|    ebook|
|       of|
|    pride|
|      and|
|prejudice|
|       by|
|     jane|
|   austen|
|         |
|     this|
|    ebook|
|       is|
|      for|
|      the|
|      use|
|       of|
|   anyone|
+---------+
only showing top 20 rows



In [20]:
words_nonull = words_clean.filter(col("word") != "")

words_nonull.show()

+---------+
|     word|
+---------+
|      the|
|  project|
|gutenberg|
|    ebook|
|       of|
|    pride|
|      and|
|prejudice|
|       by|
|     jane|
|   austen|
|     this|
|    ebook|
|       is|
|      for|
|      the|
|      use|
|       of|
|   anyone|
| anywhere|
+---------+
only showing top 20 rows



In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode, lower, regexp_extract

spark = SparkSession.builder.getOrCreate()

book = spark.read.text("/user1/1342-0.txt")

lines = book.select(split(book.value, " ").alias("line"))

words = lines.select(explode(col("line")).alias("word"))

words_lower = words.select(lower(col("word")).alias("word_lower"))

words_clean = words_lower.select(
    regexp_extract(col("word_lower"), "[a-z]*", 0).alias("word")
)

words_nonull = words_clean.where(col("word") != "")

In [31]:
#!/usr/bin/env python3

from pyspark.sql import SparkSession  # <1>

spark = SparkSession.builder.appName(  # <2>
    "Analyzing the vocabulary of Pride and Prejudice."
).getOrCreate()  # <3>

# fmt:off
# tag::sol2_1[]
from pyspark.sql.functions import col, explode

exo_2_1_df = spark.createDataFrame(
    [
        [[1, 2, 3, 4, 5]],
        [[5, 6, 7, 8, 9, 10]]
    ],
    ["numbers"]
)

solution_2_1_df = exo_2_1_df.select(explode(col("numbers")))

print(f"solution_2_1_df contains {solution_2_1_df.count()} records.")
# => solution_2_1_df contains 11 records.

solution_2_1_df.show()

solution_2_1_df contains 11 records.
+---+
|col|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
+---+



In [32]:
# tag::exo2_1[]
exo_2_1_df.show()

+-------------------+
|            numbers|
+-------------------+
|    [1, 2, 3, 4, 5]|
|[5, 6, 7, 8, 9, 10]|
+-------------------+



In [33]:

solution_2_1_df = exo_2_1_df.select(explode(col("numbers")))

# end::exo2_1[]


# tag::exo2_2[]
exo2_2_df = spark.createDataFrame(
    [["test", "more test", 10_000_000_000]], ["one", "two", "three"]
)

exo2_2_df.printSchema()


root
 |-- one: string (nullable = true)
 |-- two: string (nullable = true)
 |-- three: long (nullable = true)



In [34]:
# tag::sol2_2[]
print(len([x for x, y in exo2_2_df.dtypes if y != "string"]))  # => 1

1


In [36]:
from pyspark.sql.functions import col, length

# The `length` function returns the number of characters in a string column.

exo2_3_df = (
    spark.read.text("/user1/1342-0.txt")
    .select(length(col("value")))
    .withColumnRenamed("length(value)", "number_of_char")
)
# end::exo2_3[]

# fmt:off
# tag::sol2_3[]
exo2_3_df = (
    spark.read.text("/user1/1342-0.txt")
    .select(length(col("value")).alias("number_of_char"))
)
# end::sol2_3[]
# fmt:on

# tag::exo2_4[]


In [37]:
from pyspark.sql.functions import col, greatest

exo2_4_df = spark.createDataFrame(
    [["key", 10_000, 20_000]], ["key", "value1", "value2"]
)

exo2_4_df.printSchema()


root
 |-- key: string (nullable = true)
 |-- value1: long (nullable = true)
 |-- value2: long (nullable = true)



In [38]:
# `greatest` will return the greatest value of the list of column names,
# skipping null value

# The following statement will return an error
from pyspark.sql.utils import AnalysisException

try:
    exo2_4_mod = exo2_4_df.select(
        greatest(col("value1"), col("value2")).alias("maximum_value")
    ).select("key", "max_value")
except AnalysisException as err:
    print(err)
# end::exo2_4[]

# tag::exo2_5[]

cannot resolve 'key' given input columns: [maximum_value];
'Project ['key, 'max_value]
+- Project [greatest(value1#295L, value2#296L) AS maximum_value#300L]
   +- LogicalRDD [key#294, value1#295L, value2#296L], false



In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode, lower, regexp_extract

spark = SparkSession.builder.getOrCreate()

book = spark.read.text("/user1/1342-0.txt")

lines = book.select(split(book.value, " ").alias("line"))

words = lines.select(explode(col("line")).alias("word"))

words_lower = words.select(lower(col("word")).alias("word_lower"))

words_clean = words_lower.select(
    regexp_extract(col("word_lower"), "[a-z]*", 0).alias("word")
)

words_nonull = words_clean.where(col("word") != "")
# end::exo2_5[]

# tag::sol2_5a[]
words_without_is = words_nonull.where(col("word") != "is")
# end::sol2_5a[]

# tag::sol2_5b[]
from pyspark.sql.functions import length

words_more_than_3_char = words_nonull.where(length(col("word")) > 3)
# end::sol2_5b[]

# tag::sol2_6[]
words_no_is_not_the_if = (
    words_nonull.where(~col("word").isin(
        ["no", "is", "the", "if"])))
# end::sol2_6[]

# tag::exo2_7[]

# end::exo2_7[]

In [44]:
from pyspark.sql.functions import col, split

try:
    book = spark.read.text("/user1/1342-0.txt")
    book = book.printSchema()
    #lines = book.select(split(book.value, "").alias("line"))
    words = lines.select(explode(col("line")).alias("word"))
except AnalysisException as err:
    print(err)

root
 |-- value: string (nullable = true)



## Reference
* Data Analysis with Python and PySpark, Jonathan Rioux, February 2022 ISBN 9781617297205, Manning Publishers.